# [DataViz]: Milestone 1 (10% of the final grade)

**Group ID:** The Vizards

**Author 1 (sciper):** Salma Ed-dahabi (282284)

**Author 2 (sciper):** Antonin Faure (302686)   

**Author 3 (sciper):** Lena Vogel (297026) 

**Due date:** 07.04.2023 (11:59 pm)

[Github link] : https://github.com/com-480-data-visualization/project-2023-the-vizards

---
## Part 1 - Dataset

- We found our dataset on https://opentransportdata.swiss/de/showcase-5/ which is a website that has data on all public transports in Switzerland. As we need to import everything in order to, afterwards, filter only the data that concerns Lausanne, it is very heavy (~15GO per month).

- For the lines geojson paths we used https://openstreetmap.org data

- For the lines color we used https://t-l.ch map's data



In [1]:
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from pyproj import Proj, transform
import numpy as np
from zipfile import ZipFile
import os
from tqdm import tqdm
import json

### 1.1) Pre processing of stops

In [3]:
stops = pd.read_excel('../data/bav_list_current_timetable.xlsx', header=1).drop(index=[0])

In [4]:
lausanne_box = {
    "lat": [2529114.807, 2544879.291],
    "lon": [1159080.496, 1149116.298]
}

# Filter stops in Lausanne area
stops = stops[(stops['Coord. E'] < lausanne_box["lat"][1])
            & (stops['Coord. E'] > lausanne_box["lat"][0])
            & (stops['Coord. N'] < lausanne_box["lon"][0])
            & (stops['Coord. N'] > lausanne_box["lon"][1])
            ]

# Keep only relevant columns
stops = stops[["N° sv.85", "Nom (ordre alphab.)", "Statut", "Moyen de transport", "N° ET", "Sigle ET", "Coord. E", "Coord. N", "Altitude"]]

In [5]:
stops.head()

,N° sv.85,Nom (ordre alphab.),Statut,Moyen de transport,N° ET,Sigle ET,Coord. E,Coord. N,Altitude
890,8588453,"Aran, Pra Forny",3,Bus,#0519,TL,2544176,1150706,638
942,8501147,Archy (bif),3,NaN,#0001,SBB,2532512,1154929,404
943,8515301,Archy A (bif),3,NaN,#0001,SBB,2532896,1155064,408
2040,8501170,Bel-Air LEB,3,Zug,#0139,LEB,2536140,1158941,604
2160,8593796,"Belmont-sur-L., Arnier",3,Bus,#0519,TL,2541644,1152701,610


In [6]:
pWorld = Proj(init="epsg:4326")
pCH = Proj(init="epsg:2056")

# Convert LV95 projection to WGS84 projection
lon, lat, _ = transform(pCH,pWorld, stops["Coord. E"], stops["Coord. N"], np.zeros(stops["Coord. E"].shape))
stops["lon"] = lon
stops["lat"] = lat
stops = stops.drop(labels=["Coord. E", "Coord. N"], axis=1)
stops.rename({
    "N° sv.85": "stop_id",
    "Nom (ordre alphab.)": "stop_name",
    "Statut": "stop_status",
    "Moyen de transport": "transport_mode",
    "N° ET": "transport_id",
    "Sigle ET": "company_name",
    "Altitude": "altitude"
}, axis=1, inplace=True)

c:\Users\anton\AppData\Local\Programs\Python\Python39\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\anton\AppData\Local\Programs\Python\Python39\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\anton\AppData\Local\Temp\ipykernel_12072\1530203831.py:5: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotcha

### 1.2) Pre processing of timetables

In [7]:
keep_columns = [
    'PRODUKT_ID',
    'BETREIBER_ABK',
    'LINIEN_ID',
    'LINIEN_TEXT',
    'BPUIC',
    'FAHRT_BEZEICHNER',
    'ANKUNFTSZEIT',
    'AN_PROGNOSE',
    'AN_PROGNOSE_STATUS',
    'ABFAHRTSZEIT',
    'AB_PROGNOSE',
    'AB_PROGNOSE_STATUS'
]

map_columns = {
    'PRODUKT_ID': 'type',
    'BETREIBER_ABK': 'company',
    'LINIEN_ID': 'line_id',
    'LINIEN_TEXT': 'line_name',
    'BPUIC': 'stop_id',
    'FAHRT_BEZEICHNER': 'vehicle_id',
    'ANKUNFTSZEIT': 'arrival_time_target',
    'AN_PROGNOSE': 'arrival_time_real',
    'AN_PROGNOSE_STATUS': 'arrival_time_status',
    'ABFAHRTSZEIT': 'departure_time_target',
    'AB_PROGNOSE': 'departure_time_real',
    'AB_PROGNOSE_STATUS': 'departure_time_status'
}


#### Process timetable to extract csv data for each line

In [10]:
def lines_process(date, timetable):
    """
    Process the lines data from the provided timetable for a given date.
    Computes the arrival and departure delays for each line, merges the stop names,
    and saves the line data in separate CSV files for each line in 'data/lines' folder.

    Args:
        date (str): The date in the format 'YYYY-MM-DD'.
        timetable (pandas.DataFrame): The timetable data for the given date.

    Returns:
        None

    Raises:
        None

    """

    # Copy the relevant columns from the timetable
    line_data = timetable[['arrival_time_target', 'arrival_time_real', 'departure_time_real', 'departure_time_target', 'line_name', 'company', 'vehicle_id', 'stop_id']].copy()

    # Convert arrival and departure times to datetime format
    line_data['arrival_time_target'] = pd.to_datetime(line_data['arrival_time_target'], format='%d.%m.%Y %H:%M')
    line_data['arrival_time_real'] = pd.to_datetime(line_data['arrival_time_real'], format='%d.%m.%Y %H:%M:%S')
    line_data['departure_time_target'] = pd.to_datetime(line_data['departure_time_target'], format='%d.%m.%Y %H:%M')
    line_data['departure_time_real'] = pd.to_datetime(line_data['departure_time_real'], format='%d.%m.%Y %H:%M:%S')

    # Compute delays
    line_data['departure_delay'] = line_data.apply(lambda row: (row['departure_time_real'] - row['departure_time_target']).total_seconds() if pd.notna(row['departure_time_real']) else np.NaN, axis=1)
    line_data['arrival_delay'] = line_data.apply(lambda row: (row['arrival_time_real'] - row['arrival_time_target']).total_seconds() if pd.notna(row['arrival_time_real']) else np.NaN, axis=1)

    # Select necessary columns and merge with stop names
    line_data = line_data[['line_name', 'stop_id', 'arrival_delay', 'departure_delay']]
    line_data = line_data.merge(stops[['stop_id', 'stop_name']], on='stop_id', how='left')
    line_data['date'] = date

    # Process each line separately
    unique_lines = line_data['line_name'].unique()
    for line in unique_lines:
        line_data_specific = line_data[line_data['line_name'] == line]

        # Save line data to a CSV file
        if not os.path.isfile(f'../data/lines/{line}.csv'):
            line_data_specific.to_csv(f'../data/lines/{line}.csv', index=False)
        else:
            old_data = pd.read_csv(f'../data/lines/{line}.csv', sep=',')
            old_data = old_data[old_data['date'] != date]
            old_data = old_data.append(line_data_specific, ignore_index=True)
            old_data.to_csv(f'../data/lines/{line}.csv', index=False)
    
    # Clean up line_data DataFrame
    del line_data

### Process zip files

In [11]:
def process_zip_file(path):
    zip_file = ZipFile(path)

    activities = pd.DataFrame()
    for i, text_file in tqdm(enumerate(zip_file.infolist()), total=len(zip_file.infolist())):
        if text_file.filename.endswith('.csv') and not str(text_file.filename).startswith('__MACOSX/'):
            timetable = pd.read_csv(zip_file.open(text_file.filename), sep=";", low_memory=False)

            # Filter Lausanne area
            timetable = timetable[timetable["BPUIC"].isin(stops['stop_id'].to_list())]

            # Filter only TL
            timetable = timetable[timetable['BETREIBER_ABK'] == 'TL']

            # Keep non-cancelled trips
            timetable = timetable[timetable['FAELLT_AUS_TF'] == False]
            
            # Keep only rows with arrival or departure target
            timetable = timetable[~((timetable['ANKUNFTSZEIT'].isna()) & (timetable['ABFAHRTSZEIT'].isna()))]

            # Keep only relevant columns
            timetable = timetable[keep_columns]
            timetable.rename(map_columns, axis=1, inplace=True)
            
            date = os.path.basename(text_file.filename).split('_')[0]

            # Compute activities
            activities = pd.concat([activities, compute_stops_activity(timetable)], axis=0, ignore_index=True)
            
            # Compute lines data
            lines_process(date, timetable, False)

            del timetable

    avg_activities = activities.groupby(['stop_id', 'day_of_week', 'hour']).mean().reset_index()

    avg_activities.to_csv('../data/activities.csv', index=False)

#### One zip

In [45]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

process_zip_file('data/raw/ist-daten-2023-02.zip')

100%|██████████| 28/28 [03:04<00:00,  6.60s/it]


#### All zip in /raw folder

In [270]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# PROCESS ALL ZIP FILES IN RAW FOLDER
for month_file in os.listdir('../data/raw'):
    if month_file.endswith('.zip'):
        print('reading {} ...'.format(month_file))
        process_zip_file('../data/raw/{}'.format(month_file))

reading ist-daten-2023-03.zip ...


100%|██████████| 61/61 [07:28<00:00,  7.35s/it]


#### Compute activity by stop

In [26]:
def compute_stops_activity(df):
    """
    Compute stops activity based on the provided DataFrame.

    Args:
        df (pandas.DataFrame): The timetable data for the given date.

    Returns:
        pandas.DataFrame: DataFrame with computed stop activity.

    Raises:
        None

    Documentation:
    This function computes the activity of each stop based on the provided DataFrame.
    It extracts the day of the week and hour from the 'arrival_time_target' column,
    groups the data by stop, day of the week, and hour, and counts the number of occurrences.
    The resulting DataFrame includes the stop_id, day of the week, hour, and activity count.

    """
    # Convert 'arrival_time_target' to datetime format
    df['arrival_time_target'] = pd.to_datetime(df['arrival_time_target'], format='%d.%m.%Y %H:%M')

    # Extract day of the week and hour
    df['day_of_week'] = df['arrival_time_target'].dt.dayofweek
    df['hour'] = df['arrival_time_target'].dt.hour

    # Group by stop, day of the week, and hour, and count occurrences
    grouped = df.groupby(['stop_id', 'day_of_week', 'hour']).count().reset_index()

    # Select necessary columns and rename 'arrival_time_target' to 'activity'
    grouped = grouped[['stop_id', 'day_of_week', 'hour', 'arrival_time_target']]
    grouped = grouped.rename(columns={'arrival_time_target': 'activity'})

    return grouped

#### Compute median delay by day of week for each line

In [13]:
def compute_median_delay_by_day_of_week(line):
    """
    Compute the median delay for each stop on the specified line by day of the week.

    Args:
        line (str): Name of the line.

    Returns:
        None

    Raises:
        None

    Documentation:
    This function computes the median delay for each stop on the specified line by day of the week.
    It reads the line data from the corresponding CSV file, converts the date to datetime format,
    extracts the day of the week, and fills missing delays with zero.
    Then, it calculates the median arrival delay and median departure delay for each stop
    by grouping the data by stop ID and day of the week.
    The resulting DataFrame includes the stop_id, day of the week, median arrival delay,
    median departure delay, and stop name.
    Finally, the computed data is saved to a CSV file.
    """

    # Read line data from the corresponding CSV file
    line_data = pd.read_csv(f'lines/{line}.csv', sep=',')

    # Convert 'date' column to datetime format
    line_data['date'] = pd.to_datetime(line_data['date'], format='%Y-%m-%d')

    # Extract day of the week and fill missing delays with zero
    line_data['day_of_week'] = line_data['date'].dt.dayofweek
    line_data['arrival_delay'] = line_data['arrival_delay'].fillna(0)
    line_data['departure_delay'] = line_data['departure_delay'].fillna(0)

    # Compute median arrival_delay and median departure_delay by day of week for each stop
    delays_by_line_by_dow = line_data.groupby(['stop_id', 'day_of_week']).agg({'arrival_delay': 'median', 'departure_delay': 'median'}).reset_index()
    
    # Merge with stops DataFrame to include stop names
    line_data = delays_by_line_by_dow.merge(stops[['stop_id', 'stop_name']], on=['stop_id'], how='left')

    # Save the computed data to a CSV file
    line_data.to_csv(f'lines_delays/{line}.csv', index=False)


For all lines .csv files in 'data/lines' folder, compute the median delay by day of week: 

In [15]:
for line_file in os.listdir('../data/lines'):
    if line_file.endswith('.csv'):
        compute_median_delay_by_day_of_week(line_file.split('.')[0])

#### Compute lines championship by day of week

In [ ]:
def compute_championship_by_day_of_week():
    """
    Compute the championship rankings of delays by day of the week for all lines data in folder 'data/lines_delays' and save the results to 'data/championship.json' file.

    Args:
        None

    Returns:
        None
    """

    # Create an empty DataFrame to store the delays by line and day of the week
    delays_by_line_by_dow = pd.DataFrame()

    # Iterate through line CSV files in the 'data/lines_delays' directory
    for line in os.listdir('../data/lines_delays'):
        if line.endswith('.csv'):
            # Read line data from the corresponding CSV file
            line_data = pd.read_csv(f'../data/lines_delays/{line}', sep=',')

            # Append line data to the DataFrame
            delays_by_line_by_dow = delays_by_line_by_dow.append(line_data, ignore_index=True)
    
    # Sort the delays by day of the week, arrival delay, and departure delay in ascending order
    delays_by_line_by_dow = delays_by_line_by_dow.sort_values(['day_of_week', 'arrival_delay', 'departure_delay'], ascending=[True, True, True])

    # Rank the delays within each day of week
    delays_by_line_by_dow['arrival_delay_rank'] = delays_by_line_by_dow.groupby('day_of_week')['arrival_delay'].rank()
    delays_by_line_by_dow['departure_delay_rank'] = delays_by_line_by_dow.groupby('day_of_week')['departure_delay'].rank()
    
    # Save the computed data to a JSON file
    delays_by_line_by_dow.to_json('../data/championship.json', orient='records')

In [17]:
compute_championship_by_day_of_week()

## 1.3) Compute stops order by line

In order to compute the order of stops along the lines we use the timetable of March 24th 2023 as a reference as it's a Friday recent enough to have the same route as today.

In [22]:
def compute_lines_stop_order(date):
    timetable = pd.read_csv(f"../data/raw/{date}_istdaten.csv", sep=";", low_memory=False)

    # Filter Lausanne area
    timetable = timetable[timetable["BPUIC"].isin(stops['stop_id'].to_list())]

    # Filter only TL
    timetable = timetable[timetable['BETREIBER_ABK'] == 'TL']

    # Keep non-cancelled trips
    timetable = timetable[timetable['FAELLT_AUS_TF'] == False]

    # Keep only rows with arrival or departure target
    timetable = timetable[~((timetable['ANKUNFTSZEIT'].isna()) & (timetable['ABFAHRTSZEIT'].isna()))]

    # Keep only relevant columns
    timetable = timetable[keep_columns]
    timetable.rename(map_columns, axis=1, inplace=True)

    line_data = timetable[['arrival_time_target', 'arrival_time_real', 'departure_time_real', 'departure_time_target', 'line_name', 'company', 'vehicle_id', 'stop_id']]

    # Format dates
    line_data['arrival_time_target'] = pd.to_datetime(line_data['arrival_time_target'], format='%d.%m.%Y %H:%M')
    line_data['arrival_time_real'] = pd.to_datetime(line_data['arrival_time_real'], format='%d.%m.%Y %H:%M:%S')
    line_data['departure_time_target'] = pd.to_datetime(line_data['departure_time_target'], format='%d.%m.%Y %H:%M')
    line_data['departure_time_real'] = pd.to_datetime(line_data['departure_time_real'], format='%d.%m.%Y %H:%M:%S')

    # Fill missing arrival time with departure time
    line_data['arrival_time_target'] = line_data['arrival_time_target'].fillna(line_data['departure_time_target'])

    # Sort by arrival time target and rank (line_name, vehicle_id) by arrival time target
    line_data = line_data.sort_values(by=['arrival_time_target'])
    line_data['stop_line_idx'] = line_data.groupby(['line_name', 'vehicle_id'])['arrival_time_target'].rank()

    # Compute the size of each group
    grouped = line_data.groupby(['line_name', 'vehicle_id']).size().reset_index(name='size')

    # Sort the groups by size in descending order
    sorted_grouped = grouped.sort_values(['line_name', 'size'], ascending=[True, False])

    # Select the vehicle_id with the longest sequence for each line_name
    best_vehicle_per_line = sorted_grouped.drop_duplicates(subset='line_name')

    # Create an empty DataFrame to store the result
    result = pd.DataFrame()

    # For each line_name and vehicle_id in best_vehicle_per_line, select the corresponding rows
    for _, row in best_vehicle_per_line.iterrows():
        selected_rows = line_data[(line_data['line_name'] == row['line_name']) & (line_data['vehicle_id'] == row['vehicle_id'])]
        selected_rows = selected_rows[['line_name', 'stop_id', 'stop_line_idx']]
        selected_rows = selected_rows.merge(stops[['stop_id', 'stop_name']], on='stop_id', how='left')
        result = result.append(selected_rows)

    # Create a dictionary from the result dataframe
    result_dict = result.to_dict('records')

    # Convert the result DataFrame to a dictionary
    output_list = []

    # Iterate through the result dictionary and arrange data in the desired format
    for line_name in result['line_name'].unique():
        line_dict = {"name": line_name, "stops": []}

        line_df = result[result['line_name'] == line_name]
        for _, row in line_df.iterrows():
            line_dict['stops'].append([row['stop_line_idx'], row['stop_id'], row['stop_name']])

        output_list.append(line_dict)

    # Write the output dictionary to a JSON file
    with open('../data/lines.json', 'w') as f:
        json.dump(output_list, f)

In [23]:
compute_lines_stop_order('2023-03-24')

C:\Users\anton\AppData\Local\Temp\ipykernel_6408\3736498104.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(selected_rows)
C:\Users\anton\AppData\Local\Temp\ipykernel_6408\3736498104.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(selected_rows)
C:\Users\anton\AppData\Local\Temp\ipykernel_6408\3736498104.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(selected_rows)
C:\Users\anton\AppData\Local\Temp\ipykernel_6408\3736498104.py:56: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(selected_rows)
C:\Users\anton\AppData\Local\Temp\ipyker

,line_name,stop_id,stop_line_idx,stop_name
0,1,8592068,1.0,"Lausanne, Maladière"
1,1,8592080,2.0,"Lausanne, Montoie"
2,1,8592110,3.5,"Lausanne, Riant-Cour"
3,1,8591983,3.5,"Lausanne, Batelière"
4,1,8592005,5.0,"Lausanne, Cèdres"
...,...,...,...,...
10,m1,8501215,11.0,"Ecublens VD, Bassenges"
11,m1,8501216,12.0,"Ecublens VD, Cerisaie"
12,m1,8501217,13.0,"Chavannes-R., Crochy"
13,m1,8501218,14.0,"Ecublens VD, Epenex"


## 1.4) Output data to GeoJSON format

In [18]:
import json

def create_square_polygon(center_lon, center_lat, offset=0.00015):
    return [
        [center_lon - offset, center_lat - offset],
        [center_lon + offset, center_lat - offset],
        [center_lon + offset, center_lat + offset],
        [center_lon - offset, center_lat + offset],
        [center_lon - offset, center_lat - offset]
    ]

def lines_to_geojson():
    # for each file in lines_delays folder append the features to geojson file
    geojson = {
        "type": "FeatureCollection",
        "features": []
    }

    for line_file in os.listdir('../data/lines_delays'):
        if line_file.endswith('.csv'):
            line_data = pd.read_csv(f'../data/lines_delays/{line_file}', sep=',')
            line_data['stop_id'] = line_data['stop_id']
            #line_data['stop_line_idx'] = line_data['stop_line_idx'].astype(str)
            line_data['day_of_week'] = line_data['day_of_week'].astype(str)
            line_data['arrival_delay'] = line_data['arrival_delay'].fillna(0)
            line_data['departure_delay'] = line_data['departure_delay'].fillna(0)
            line_data['stop_name'] = line_data['stop_name'].astype(str)

            for i, row in line_data.iterrows():
                stop_coordinates = stops[stops['stop_id'] == row['stop_id']].iloc[0]
                polygon_coordinates = create_square_polygon(stop_coordinates['lon'], stop_coordinates['lat'])

                geojson['features'].append({
                    "type": "Feature",
                    "geometry": {
                        "type": "Polygon",
                        "coordinates": [polygon_coordinates]
                    },
                    "properties": {
                        "stop_id": row['stop_id'],
                        "day_of_week": row['day_of_week'],
                        "arrival_delay": row['arrival_delay'],
                        "departure_delay": row['departure_delay'],
                        "stop_name": row['stop_name'],
                        "line_name": line_file.split('.')[0]
                    }
                })

    with open('../data/lines_delays.geojson', 'w') as outfile:
        json.dump(geojson, outfile)


In [ ]:
lines_to_geojson()

In [50]:
def stops_to_geojson():
    # for each file in lines_delays folder append the features to geojson file
    geojson = {
        "type": "FeatureCollection",
        "features": []
    }

    stops_data = pd.read_csv(f'../data/activities.csv', sep=',')
    stops_data['stop_id'] = stops_data['stop_id']
    stops_data['day_of_week'] = stops_data['day_of_week'].astype(str)
    stops_data['hour'] = stops_data['hour'].astype(str)
    
    for i, row in stops_data.iterrows():
        stop_coordinates = stops[stops['stop_id'] == row['stop_id']].iloc[0]
        stop_name = stops[stops['stop_id'] == row['stop_id']].iloc[0]['stop_name']

        geojson['features'].append({
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [stop_coordinates['lon'], stop_coordinates['lat']]
            },
            "properties": {
                "stop_id": row['stop_id'],
                "day_of_week": row['day_of_week'],
                "hour": row['hour'],
                "stop_name": stop_name,
                "activity": row['activity']
            }
        })

    with open('../data/stops.geojson', 'w') as outfile:
        json.dump(geojson, outfile)

In [51]:
stops_to_geojson()

---
## PART 2 - Problematic

**Objectives overview**:

Frame the general topic of your visualization and the main axis that you want to develop.

**1)** What am I trying to show with my visualization? With our visualisations we would like to show: 
- Lausanne traffic depending on the day and time: the punctuality of the buses is a good estimation of the traffic.
- Lausanne connectivity through the city and its suburbs: seing the concentration of the public transports (and not only the lines/trajectories) allows anyone to judge whether a region is well served or not.

**2)** Think of an overview for the project, your motivation, and the target audience.
- The main target audience is people who would like to move to Lausanne in a near future.
- Another audience could be the TL themselves, as it is useful to know what are the problematic areas, which make the buses late. In general, it could be used in an analytic way for other entities such as geographists or sociologists

---
## Part 3 - Exploratory Data Analysis

Pre-processing of the data set you chose.

Show some basic statistics and get insights about the data



In [12]:
fig = px.scatter_mapbox(
    stops, lat="lat",
    lon="lon",
    color="company_name", # which column to use to set the color of markers
    hover_name="stop_name", # column added to hover information,
    zoom=3, mapbox_style='open-street-map', height=800
    )

fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        bearing=0,
        center=dict(lat=46.52097182546228, lon=6.633647865079138),
        pitch=0,
        zoom=11
    ),
    title="Public transports stops in Lausanne",
)
fig.show()

In [13]:
fig = px.histogram(stops, x="company_name", color="company_name", title="Number of stops for each public transport company")
fig.show()

In [26]:
def plot_activity_by_line(date):
    # Read the data
    timetable = pd.read_csv(f"../data/raw/{date}_istdaten.csv", sep=";", low_memory=False)

    # Filter Lausanne area
    timetable = timetable[timetable["BPUIC"].isin(stops['stop_id'].to_list())]

    # Filter only TL
    timetable = timetable[timetable['BETREIBER_ABK'] == 'TL']

    # Keep non-cancelled trips
    timetable = timetable[timetable['FAELLT_AUS_TF'] == False]

    # Keep only rows with arrival or departure target
    timetable = timetable[~((timetable['ANKUNFTSZEIT'].isna()) & (timetable['ABFAHRTSZEIT'].isna()))]

    # Keep only relevant columns
    timetable = timetable[keep_columns]
    timetable.rename(map_columns, axis=1, inplace=True)

    line_data = timetable[['line_name', 'company', 'vehicle_id', 'stop_id']]

    # Compute number of records per line_name 
    grouped = line_data.groupby(['line_name']).count().reset_index()

    # rename columns
    grouped.rename({'vehicle_id': 'count'}, axis=1, inplace=True)

    # Plot the number of records per line_name
    fig = px.bar(grouped, x='line_name', y='count', title=f"Number of records per line on {date}")
    fig.show()

In [27]:
plot_activity_by_line('2023-03-24')

In [36]:
def plot_median_delay_by_line(date):
    # Read the data
    timetable = pd.read_csv(f"../data/raw/{date}_istdaten.csv", sep=";", low_memory=False)

    # Filter Lausanne area
    timetable = timetable[timetable["BPUIC"].isin(stops['stop_id'].to_list())]

    # Filter only TL
    timetable = timetable[timetable['BETREIBER_ABK'] == 'TL']

    # Keep non-cancelled trips
    timetable = timetable[timetable['FAELLT_AUS_TF'] == False]

    # Keep only rows with arrival or departure target
    timetable = timetable[~((timetable['ANKUNFTSZEIT'].isna()) & (timetable['ABFAHRTSZEIT'].isna()))]

    # Keep only relevant columns
    timetable = timetable[keep_columns]
    timetable.rename(map_columns, axis=1, inplace=True)

    # Convert 'arrival_time_target' and 'arrival_time_real' to datetime format
    timetable['arrival_time_target'] = pd.to_datetime(timetable['arrival_time_target'], format='%d.%m.%Y %H:%M')
    timetable['arrival_time_real'] = pd.to_datetime(timetable['arrival_time_real'], format='%d.%m.%Y %H:%M:%S')

    # Compute arrival delay as the difference between real and target arrival times
    timetable['arrival_delay'] = (timetable['arrival_time_real'] - timetable['arrival_time_target']).dt.total_seconds()

    # Convert 'departure_time_target' and 'departure_time_real' to datetime format
    timetable['departure_time_target'] = pd.to_datetime(timetable['departure_time_target'], format='%d.%m.%Y %H:%M')
    timetable['departure_time_real'] = pd.to_datetime(timetable['departure_time_real'], format='%d.%m.%Y %H:%M:%S')

    # Compute departure delay as the difference between real and target departure times
    timetable['departure_delay'] = (timetable['departure_time_real'] - timetable['departure_time_target']).dt.total_seconds()

    # Extract day of the week from 'arrival_time_target'
    timetable['day_of_week'] = timetable['arrival_time_target'].dt.dayofweek

    # Compute median arrival and departure delays per line
    median_delays_by_line = timetable.groupby('line_name').agg({'arrival_delay': 'median', 'departure_delay': 'median'}).reset_index()

    # Melt the DataFrame to combine arrival and departure delays into a single column
    melted_delays = pd.melt(median_delays_by_line, id_vars='line_name', value_vars=['arrival_delay', 'departure_delay'], var_name='delay_type', value_name='median_delay')

    # Plotting
    fig = px.bar(melted_delays, x='line_name', y='median_delay', color='delay_type',
                 labels={'line_name': 'Line', 'median_delay': 'Median Delay', 'delay_type': 'Delay Type'},
                 title=f'Median Arrival and Departure Delays by Line - {date}')
    fig.show()

In [37]:
plot_median_delay_by_line('2023-03-24')

---
## Part 4 - Related work

- What others have already done with the data?

We took our data from the website https://opentransportdata.swiss/de/showcase-5/, which already labels and organize its data.

- Why is your approach original?

We would like to concentrate our visualizations only on Lausanne, to allow any Lausanne habitant or future habitant to have access to a full view of the public transports of the city

- What source of inspiration do you take? Visualizations that you found on other websites or magazines (might be unrelated to your data).

Other visualisations of that type have been done, for instance in Switzerland: https://observablehq.com/@alexmasselot/mapping-swiss-trains-delays-over-one-day/2, which shows the delays of the trains in Switzerland, and https://mobility.portal.geops.io/world.geops.transit?baselayer=world.geops, which displays most of the public transports in the world and their position in real time.

- In case you are using a dataset that you have already explored in another context (ML or ADA course, semester project...), you are required to share the report of that work to outline the differences with the submission for this class.

N/A

